In [1]:
import os

import pandas as pd
from dotenv import load_dotenv
from sqlalchemy import create_engine,inspect

In [ ]:
load_dotenv()

DB_HOST = os.getenv("DB_HOST")
DB_PORT = os.getenv("DB_PORT")
DB_USER = os.getenv("DB_USER")
DB_PASSWORD = os.getenv("DB_PASSWORD")
DB_NAME = os.getenv("DB_NAME")
MART_SCHEMA = "zookeepers_mart"
PRODUCTION_SCHEMA = "zookeepers_production"

In [ ]:
def get_engine_connection():
        """Connects to postgreSQL DBMS on AWS Aurora

        Returns:
            DB engine
        """
        conn_string = f"postgresql+psycopg2://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}"

        return create_engine(conn_string)

engine = get_engine_connection()

In [ ]:
query = f"""
    CREATE TABLE IF NOT EXISTS {MART_SCHEMA}.recent_rides (
        user_id SMALLINT NOT NULL,
        ride_id SMALLINT NOT NULL,
        gender VARCHAR(6) NOT NULL,
        age INT NOT NULL,
        begin_timestamp TIMESTAMP NOT NULL,
        total_duration_sec SMALLINT NOT NULL,
        total_power DECIMAL NOT NULL, 
        mean_power DECIMAL NOT NULL,
        mean_resistance DECIMAL NOT NULL, 
        mean_rpm DECIMAL NOT NULL,
        mean_heart_rate DECIMAL NOT NULL
    )
"""

engine.execute(query)

In [ ]:
query = f"""
    INSERT INTO {MART_SCHEMA}.recent_rides

    WITH recent_rides_data AS(

        WITH user_gender_dob AS (
        SELECT user_id, gender,
            DATEDIFF(hour,date_of_birth, CURRENT_DATE)/8766 AS age
            FROM {PRODUCTION_SCHEMA}.users
        ),

        rides_before AS (
            SELECT *
                FROM {PRODUCTION_SCHEMA}.rides
                WHERE TO_DATE(rs.begin_timestamp) > DATEADD(HOUR, -12, CURRENT_DATE)
        )

        SELECT ugd.*, rb.*
            FROM user_gender_dob AS ugd
            RIGHT JOIN rides_before AS rb
                ON ugd.user_id = rb.user_id
    )

    SELECT * FROM recent_rides_data;
"""

engine.execute(query)